In [1]:
import sys
sys.path.insert(1, '../scr')
from DictionaryReader import PONSReader, PhraseReader
from toolbox import Configurator, AnkiCommunicator, AnkiCardWriter
from ExerciseWriter import ExerciseWriter
from Exercise import ExerciseFactory, ExerciseGatherer, ExampleSentences
from pathlib import Path
import shutil
import pyperclip
import json

def copy_folder(source_folder, destination_folder) -> None:
    destination_path = Path(destination_folder)
    source_path = Path(source_folder)
    if not destination_path.exists():
        shutil.copytree(source_path, destination_path)

### Initialisation

In [2]:
# User inputs
date = 20240602


# Make a new folder
template_folder = '../scr/resources/Templates'
export_folder = f'./Exports/{date}'
copy_folder(template_folder, export_folder)

def_template_path = '/Definition/template.tex'
gap_filling_template_path = '/Fill-in-the-gap/template.tex'
dialogue_template_path = '/Dialogue/template.tex'

def_output_folder = export_folder + '/Definition'
gap_filling_output_folder = export_folder + '/Fill-in-the-gap'
dialogue_output_folder = export_folder + '/Dialogue'

factory = ExerciseFactory()
def_gatherer = ExerciseGatherer()
def_writer = ExerciseWriter(gatherer=def_gatherer, template_path=def_template_path, output_folder=def_output_folder)
gap_filling_gatherer = ExerciseGatherer()
gap_filling_exercise_writer = ExerciseWriter(gatherer=gap_filling_gatherer, template_path=gap_filling_template_path, output_folder=gap_filling_output_folder)
dialogue_gatherer = ExerciseGatherer()
dialogue_exercise_writer = ExerciseWriter(gatherer=dialogue_gatherer, template_path=dialogue_template_path, output_folder=dialogue_output_folder)

In [6]:
category = 'German - Vocab Builder'
# category = 'Everyday German'
num_of_words_to_learn = 10

configurator = Configurator(json_path=f'../scr/config/{category}/config.json')

tmp_dict = dict()
with open('tmp.json', 'w') as file:
    json.dump(tmp_dict, file)

# Get the words to learn and review for tomorrow
tomorrow_new = configurator.get_n_words_to_learn(num_of_words_to_learn)
tmp_dict['tomorrow new'] = tomorrow_new

anki = AnkiCommunicator()
tomorrow_review = anki.get_words_for_tomorrow(deck_name=category, item='Front')
tmp_dict['tomorrow review'] = tomorrow_review


tomorrow_all = tomorrow_new + tomorrow_review
print(f'''
Number of all words: {len(tomorrow_all)};
Number of new words: {len(tomorrow_new)};
Number of old words: {len(tomorrow_review)}.
''')
with open('tmp.json', 'w') as file:
    json.dump(tmp_dict, file, indent=4, ensure_ascii=False)


Number of all words: 10;
Number of new words: 10;
Number of old words: 0.



### Definition

#### Set the word list and the set index here and get the prompt to create definitions

In [7]:
def_start_idx = 1
def_end_idx = 10
def_set_index = 1

def_word_list = tomorrow_new[def_start_idx-1:def_end_idx]
definitions_set = factory.create_exercise(exercise_type='Definition', word_list=def_word_list)
definitions_set.import_definition_from_dictionary()
definitions_set.finish_import()
def_gatherer.import_exercise(definitions_set)
def_writer.render_template(set_index=def_set_index)

The word Gemälde does not exist in the dictionary.
The word föhnen does not exist in the dictionary.
The word Tagesablauf does not exist in the dictionary.
The word Staatsdienst does not exist in the dictionary.
The word bewundern does not exist in the dictionary.
The word Wirtschaftszweig does not exist in the dictionary.
The word Betrieb does not exist in the dictionary.
The word verwöhnen does not exist in the dictionary.
The word Skulptur does not exist in the dictionary.
The word Trompete does not exist in the dictionary.


### Vocabulary Exercises

#### Set the word list here and get the prompt to create example sentences

In [ ]:
vocab_start_idx = 16
vocab_end_idx = 35
repetition = 'two'

# vocab_word_list = tomorrow_new[vocab_start_idx-1:vocab_end_idx]
vocab_word_list = tomorrow_review[vocab_start_idx-1:vocab_end_idx]

example_sentences = ExampleSentences(vocab_word_list)
example_sentences.create_prompt(number_of_sentences=repetition)
example_sentences.get_prompt()

#### Paste the example sentences here

In [ ]:
sentences = pyperclip.paste()
example_sentences.import_sentences(text=sentences)
example_sentences.finish_import()

##### Enter the set index here

In [ ]:
gap_filling_set_index = 4

gap_filling_exercise = factory.create_exercise(exercise_type='Fill in the gap', word_list=example_sentences.word_list, example_sentences=example_sentences)
gap_filling_exercise.finish_import()
gap_filling_gatherer.import_exercise(exercise=gap_filling_exercise)
gap_filling_exercise_writer.render_template(set_index=gap_filling_set_index)

### Configure the reader

In [ ]:
reader = PONSReader(word_list=tomorrow_new)
# reader = PhraseReader(word_list=tomorrow_new)

anki_writer = AnkiCardWriter(word_entry_list=reader.get_word_entry_list())
anki_writer.write_cards(csv_path=f'Exports/{category}_{date}.csv')

In [ ]:
configurator.study_n_words(num_of_words_to_learn)